In [1]:
import requests as r
import os
from twilio.rest import Client

#constants
stock_name = 'TSLA'
company_name = 'Tesla Inc.'
stock_endpoint = 'https://www.alphavantage.co/query'
news_endpoint = 'https://newsapi.org/v2/everything'
stock_api_key = ''
news_api_key = ''
#twilio constants
twilio_sid = ''
twilio_auth_token = ''
twilio_virtual_number = ''
twilio_actual_number = ''

stock_parameters = {
    'function': 'TIME_Series_Daily',
    'symbol': stock_name,
    'apikey': stock_api_key,
}

stock_response = r.get(stock_endpoint, params = stock_parameters)
stock_data = stock_response.json()
data = stock_data['Time Series (Daily)']
#print(data)
#list comprehension - data_list = [new_item for item in list]
#.items() method - method returns a view object
#view object - contains the key-value pairs of the dictionary, as tuples in a list
data_list = [value for (key,value) in data.items()]
#print(data_list)
y_data =  data_list[0]
#print(y_data)
#yesterdays closing price - closing_y_data
closing_y_data = y_data['4. close']
#print(closing_y_data)

dby_data = data_list[1]
#day before yesterday's closing price - closing_dby_data
closing_dby_data = dby_data['4. close']
#print(closing_dby_data)

price_difference = float(closing_y_data) - float(closing_dby_data)
#stylistic additions
up_down = None
if price_difference > 0:
    up_down = '🔺'
else:
    up_down = '🔻'
#print(price_difference)
percent_difference = round((price_difference/float(closing_y_data)) * 100)
#print(percent_difference)

#abs - Return the absolute value of a number
if abs(percent_difference) > 1:
    #create news parameters
    news_parameters = {
        'apikey': news_api_key,
        'qInTitle': company_name
    }
    
    news_response = r.get(news_endpoint, params = news_parameters)
    #print(news_response)
    #<Response [200]> - no issues with the code
    #print(news_response.json())
    news_articles = news_response.json()['articles']
    #print(news_articles)
    #grab the first 3 articles
    relevant_articles = news_articles[:3]
    #print(relevant_articles)
    format_articles = [f"{stock_name}: {up_down}{percent_difference}%\nHeadline: {article['title']}.\nBrief: {article['description']}" for article in relevant_articles]
    client = Client(twilio_sid, twilio_auth_token)
    for article in format_articles:
        message = client.messages.create(
            body = article,
            from_ = '',
            to = ''
        )
        print(message.status)   

KeyError: 'Time Series (Daily)'